# Modern Pandas (Part 1)

* see: https://github.com/TomAugspurger/effective-pandas
* see: https://tomaugspurger.github.io/modern-1-intro

## Get the Data

In [2]:
import os
import zipfile

import requests
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import requests

In [26]:
# filter the warning for now on
import warnings
warnings.simplefilter("ignore", DeprecationWarning)

In [3]:
headers = {
    'Referer': 'https://www.transtats.bts.gov/DL_SelectFields.asp?Table_ID=236&DB_Short_Name=On-Time',
    'Origin': 'https://www.transtats.bts.gov',
    'Content-Type': 'application/x-www-form-urlencoded',
}

params = (
    ('Table_ID', '236'),
    ('Has_Group', '3'),
    ('Is_Zipped', '0'),
)

In [8]:
with open('modern-1-url.txt', encoding='utf-8') as f:
    data = f.read().strip()
    
os.makedirs('data', exist_ok=True)
dest = "data/flights.csv.zip"

In [11]:
if not os.path.exists(dest):
    r = requests.post('https://www.transtats.bts.gov/DownLoad_Table.asp', headers=headers, params=params, data=data, stream=True)
    
    with open(dest, 'wb') as f:
        for chunk in r.iter_content(chunk_size=102400): 
            if chunk:
                f.write(chunk)

In [18]:
zf = zipfile.ZipFile(dest)
fp = zf.extract(zf.filelist[0].filename, path='data/')

In [20]:
df = pd.read_csv(fp, parse_dates=['FL_DATE']).rename(columns=str.lower)

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 450017 entries, 0 to 450016
Data columns (total 33 columns):
fl_date                  450017 non-null datetime64[ns]
unique_carrier           450017 non-null object
airline_id               450017 non-null int64
tail_num                 449378 non-null object
fl_num                   450017 non-null int64
origin_airport_id        450017 non-null int64
origin_airport_seq_id    450017 non-null int64
origin_city_market_id    450017 non-null int64
origin                   450017 non-null object
origin_city_name         450017 non-null object
dest_airport_id          450017 non-null int64
dest_airport_seq_id      450017 non-null int64
dest_city_market_id      450017 non-null int64
dest                     450017 non-null object
dest_city_name           450017 non-null object
crs_dep_time             450017 non-null int64
dep_time                 441476 non-null float64
dep_delay                441476 non-null float64
taxi_out                

# Indexing

## Slicing

In [28]:
df.ix[10:15, ['fl_date', 'tail_num']] # DeprecationWarning!

,fl_date,tail_num
10,2017-01-01,N855AA
11,2017-01-01,N785AA
12,2017-01-01,N785AA
13,2017-01-01,N790AA
14,2017-01-01,N784AA
15,2017-01-01,N783AA


In [30]:
df.groupby(['airline_id'])['fl_date', 'unique_carrier'].first()

,fl_date,unique_carrier
airline_id,,
19393,2017-01-01,WN
19690,2017-01-01,HA
19790,2017-01-01,DL
19805,2017-01-01,AA
19930,2017-01-01,AS
19977,2017-01-01,UA
20304,2017-01-01,OO
20366,2017-01-01,EV
20409,2017-01-01,B6


In [51]:
first = df.groupby(['airline_id'])['fl_date', 'unique_carrier'].first()

In [53]:
first.ix[10:15, ['fl_date', 'unique_carrier']]

,fl_date,unique_carrier
airline_id,,


In [54]:
first.index

Int64Index([19393, 19690, 19790, 19805, 19930, 19977, 20304, 20366, 20409,
            20416, 20436, 21171],
           dtype='int64', name='airline_id')

In [55]:
first.ix[20436:21171, ['fl_date', 'unique_carrier']]

,fl_date,unique_carrier
airline_id,,
20436,2017-01-01,F9
21171,2017-01-01,VX


In [56]:
first.iloc[10:15, [0, 1]]

,fl_date,unique_carrier
airline_id,,
20436,2017-01-01,F9
21171,2017-01-01,VX


In [57]:
first.loc[20436:21171, ['fl_date', 'unique_carrier']]

,fl_date,unique_carrier
airline_id,,
20436,2017-01-01,F9
21171,2017-01-01,VX


In [59]:
first = df.groupby('unique_carrier').first()

In [60]:
first

,fl_date,airline_id,tail_num,fl_num,origin_airport_id,origin_airport_seq_id,origin_city_market_id,origin,origin_city_name,dest_airport_id,...,arr_time,arr_delay,cancelled,cancellation_code,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,unnamed: 32
unique_carrier,,,,,,,,,,,,,,,,,,,,,
AA,2017-01-01,19805,N787AA,1,12478,1247803,31703,JFK,"New York, NY",12892,...,1209.0,27.0,0.0,A,27.0,0.0,0.0,0.0,0.0,NaN
AS,2017-01-01,19930,N303AS,360,14747,1474703,30559,SEA,"Seattle, WA",14893,...,1703.0,48.0,1.0,A,0.0,0.0,48.0,0.0,0.0,NaN
B6,2017-01-01,20409,N593JB,264,14747,1474703,30559,SEA,"Seattle, WA",12478,...,506.0,1.0,0.0,A,0.0,0.0,0.0,0.0,54.0,NaN
DL,2017-01-01,19790,N942DL,2,12478,1247803,31703,JFK,"New York, NY",11697,...,2301.0,5.0,0.0,B,16.0,0.0,1.0,0.0,0.0,NaN
EV,2017-01-01,20366,N12530,4101,12915,1291503,31205,LCH,"Lake Charles, LA",12266,...,634.0,-1.0,0.0,A,14.0,0.0,15.0,0.0,26.0,NaN
F9,2017-01-01,20436,N201FR,1462,10397,1039705,30397,ATL,"Atlanta, GA",12953,...,807.0,-23.0,0.0,A,0.0,0.0,36.0,0.0,0.0,NaN
HA,2017-01-01,19690,N378HA,1,12892,1289204,32575,LAX,"Los Angeles, CA",12173,...,1108.0,8.0,0.0,A,457.0,0.0,0.0,0.0,0.0,NaN
NK,2017-01-01,20416,N664NK,174,11697,1169704,32467,FLL,"Fort Lauderdale, FL",12953,...,1904.0,-25.0,0.0,A,0.0,0.0,40.0,0.0,0.0,NaN
OO,2017-01-01,20304,N920SW,5456,14771,1477102,32457,SFO,"San Francisco, CA",13264,...,2013.0,-6.0,0.0,B,0.0,0.0,0.0,0.0,17.0,NaN


In [61]:
first.index

Index(['AA', 'AS', 'B6', 'DL', 'EV', 'F9', 'HA', 'NK', 'OO', 'UA', 'VX', 'WN'], dtype='object', name='unique_carrier')

In [62]:
first.ix[10:15, ['fl_date', 'tail_num']]

,fl_date,tail_num
unique_carrier,,
VX,2017-01-01,N846VA
WN,2017-01-01,N955WN


In [63]:
first.loc[['VX', 'WIN'], ['fl_date', 'tail_num']]

/Users/yuki/.pyenv/versions/miniconda3-4.1.11/envs/ml_env/lib/python3.6/site-packages/ipykernel/__main__.py:1: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  if __name__ == '__main__':


,fl_date,tail_num
unique_carrier,,
VX,2017-01-01,N846VA
WIN,NaT,NaN


In [68]:
first.loc[['AA','AS','DL'], ['fl_date', 'airline_id']]

,fl_date,airline_id
unique_carrier,,
AA,2017-01-01,19805
AS,2017-01-01,19930
DL,2017-01-01,19790


In [67]:
first.iloc[[0,1,3], [0,1]]

,fl_date,airline_id
unique_carrier,,
AA,2017-01-01,19805
AS,2017-01-01,19930
DL,2017-01-01,19790


## SettingWithCopy 

In [70]:
f = pd.DataFrame({'a':[1,2,3,4,5], 'b':[10,20,30,40,50]})

In [71]:
f

,a,b
0,1,10
1,2,20
2,3,30
3,4,40
4,5,50


In [72]:
f.index

RangeIndex(start=0, stop=5, step=1)

In [77]:
f[f['a'] <= 3]['b'] = f[f['a'] <= 3 ]['b'] / 10

/Users/yuki/.pyenv/versions/miniconda3-4.1.11/envs/ml_env/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [78]:
f

,a,b
0,1,10
1,2,20
2,3,30
3,4,40
4,5,50


In [79]:
with pd.option_context('mode.chained_assignment', None):
    f[f['a'] <= 3]['b'] = f[f['a'] <= 3 ]['b'] / 10

In [80]:
f

,a,b
0,1,10
1,2,20
2,3,30
3,4,40
4,5,50


In [83]:
f.loc[f['a'] <= 3, 'b'] = f.loc[f['a'] <= 3, 'b'] / 10

In [84]:
f

,a,b
0,1,1.0
1,2,2.0
2,3,3.0
3,4,40.0
4,5,50.0


## Multidimensional Indexing

In [88]:
df.index

RangeIndex(start=0, stop=450017, step=1)

In [87]:
df[df.columns[:4]].head()

,fl_date,unique_carrier,airline_id,tail_num
0,2017-01-01,AA,19805,N787AA
1,2017-01-01,AA,19805,N783AA
2,2017-01-01,AA,19805,N791AA
3,2017-01-01,AA,19805,N391AA
4,2017-01-01,AA,19805,N346AA


In [91]:
df.head()

,fl_date,unique_carrier,airline_id,tail_num,fl_num,origin_airport_id,origin_airport_seq_id,origin_city_market_id,origin,origin_city_name,...,arr_time,arr_delay,cancelled,cancellation_code,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,unnamed: 32
0,2017-01-01,AA,19805,N787AA,1,12478,1247803,31703,JFK,"New York, NY",...,1209.0,27.0,0.0,NaN,27.0,0.0,0.0,0.0,0.0,NaN
1,2017-01-01,AA,19805,N783AA,2,12892,1289204,32575,LAX,"Los Angeles, CA",...,1809.0,42.0,0.0,NaN,34.0,0.0,8.0,0.0,0.0,NaN
2,2017-01-01,AA,19805,N791AA,4,12892,1289204,32575,LAX,"Los Angeles, CA",...,2040.0,42.0,0.0,NaN,7.0,0.0,0.0,0.0,35.0,NaN
3,2017-01-01,AA,19805,N391AA,5,11298,1129804,30194,DFW,"Dallas/Fort Worth, TX",...,1749.0,97.0,0.0,NaN,77.0,0.0,20.0,0.0,0.0,NaN
4,2017-01-01,AA,19805,N346AA,6,13830,1383002,33830,OGG,"Kahului, HI",...,642.0,42.0,0.0,NaN,0.0,0.0,42.0,0.0,0.0,NaN


In [89]:
hdf = df.set_index(['unique_carrier', 'origin', 'dest', 'tail_num', 'fl_date']).sort_index)

In [90]:
hdf.head()

airline_id  fl_num  \
unique_carrier origin dest tail_num fl_date                          
AA             ABQ    DFW  N3ABAA   2017-01-15       19805    2611   
                                    2017-01-29       19805    1282   
                           N3AEAA   2017-01-11       19805    2511   
                           N3AJAA   2017-01-24       19805    2511   
                           N3AVAA   2017-01-11       19805    1282   

                                                origin_airport_id  \
unique_carrier origin dest tail_num fl_date                         
AA             ABQ    DFW  N3ABAA   2017-01-15              10140   
                                    2017-01-29              10140   
                           N3AEAA   2017-01-11              10140   
                           N3AJAA   2017-01-24              10140   
                           N3AVAA   2017-01-11              10140   

                                                origin_airport_seq_id  \
unique_carrier origin dest tail_num fl_date                             
AA             ABQ    DFW  N3ABAA   2017-01-15                1014003   
                                    2017-01-29                1014003   
                           N3AEAA   2017-01-11                1014003   
                           N3AJAA   2017-01-24                1014003   
                           N3AVAA   2017-01-11                1014003   

                                                origin_city_market_id  \
unique_carrier origin dest tail_num fl_date                             
AA             ABQ    DFW  N3ABAA   2017-01-15                  30140   
                                    2017-01-29                  30140   
                           N3AEAA   2017-01-11                  30140   
                           N3AJAA   2017-01-24                  30140   
                           N3AVAA   2017-01-11                  30140   

                                               origin_city_name  \
unique_carrier origin dest tail_num fl_date                       
AA             ABQ    DFW  N3ABAA   2017-01-15  Albuquerque, NM   
                                    2017-01-29  Albuquerque, NM   
                           N3AEAA   2017-01-11  Albuquerque, NM   
                           N3AJAA   2017-01-24  Albuquerque, NM   
                           N3AVAA   2017-01-11  Albuquerque, NM   

                                                dest_airport_id  \
unique_carrier origin dest tail_num fl_date                       
AA             ABQ    DFW  N3ABAA   2017-01-15            11298   
                                    2017-01-29            11298   
                           N3AEAA   2017-01-11            11298   
                           N3AJAA   2017-01-24            11298   
                           N3AVAA   2017-01-11            11298   

                                                dest_airport_seq_id  \
unique_carrier origin dest tail_num fl_date                           
AA             ABQ    DFW  N3ABAA   2017-01-15              1129804   
                                    2017-01-29              1129804   
                           N3AEAA   2017-01-11              1129804   
                           N3AJAA   2017-01-24              1129804   
                           N3AVAA   2017-01-11              1129804   

                                                dest_city_market_id  \
unique_carrier origin dest tail_num fl_date                           
AA             ABQ    DFW  N3ABAA   2017-01-15                30194   
                                    2017-01-29                30194   
                           N3AEAA   2017-01-11                30194   
                           N3AJAA   2017-01-24                30194   
                           N3AVAA   2017-01-11                30194   

                                                       dest_city_name  \
unique_carrier origin dest tail_num fl_d

In [95]:
hdf.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 450017 entries, (AA, ABQ, DFW, N3ABAA, 2017-01-15 00:00:00) to (WN, TUS, SAN, N957WN, 2017-01-29 00:00:00)
Data columns (total 28 columns):
airline_id               450017 non-null int64
fl_num                   450017 non-null int64
origin_airport_id        450017 non-null int64
origin_airport_seq_id    450017 non-null int64
origin_city_market_id    450017 non-null int64
origin_city_name         450017 non-null object
dest_airport_id          450017 non-null int64
dest_airport_seq_id      450017 non-null int64
dest_city_market_id      450017 non-null int64
dest_city_name           450017 non-null object
crs_dep_time             450017 non-null int64
dep_time                 441476 non-null float64
dep_delay                441476 non-null float64
taxi_out                 441244 non-null float64
wheels_off               441244 non-null float64
wheels_on                440746 non-null float64
taxi_in                  440746 non-null float

In [96]:
hdf.loc[['AA', 'DL', 'US'], ['dep_time', 'dep_delay']]

dep_time  dep_delay
unique_carrier origin dest tail_num fl_date                        
AA             ABQ    DFW  N3ABAA   2017-01-15     500.0        0.0
                                    2017-01-29     757.0       -3.0
                           N3AEAA   2017-01-11    1451.0       -9.0
                           N3AJAA   2017-01-24    1502.0        2.0
                           N3AVAA   2017-01-11     752.0       -8.0
                           N3AWAA   2017-01-27    1550.0       50.0
                           N3AXAA   2017-01-16    1524.0       24.0
                                    2017-01-17     757.0       -3.0
                           N3BJAA   2017-01-25     823.0       23.0
                           N3BPAA   2017-01-11    1638.0       -7.0
                           N3BTAA   2017-01-26     753.0       -7.0
                           N3BYAA   2017-01-18    1452.0       -8.0
                           N3CAAA   2017-01-23     453.0       -7.0
                           N3CBAA   2017-01-13    1456.0       -4.0
                           N3CDAA   2017-01-12    1455.0       -5.0
                                    2017-01-28     758.0       -2.0
                           N3CEAA   2017-01-21     455.0       -5.0
                           N3CGAA   2017-01-18     759.0       -1.0
                           N3CWAA   2017-01-27    1638.0       -7.0
                           N3CXAA   2017-01-31     752.0       -8.0
                           N3DBAA   2017-01-19    1637.0       -8.0
                           N3DMAA   2017-01-13    1638.0       -7.0
                           N3DRAA   2017-01-27     753.0       -7.0
                           N3DVAA   2017-01-09    1636.0       -9.0
                           N3DYAA   2017-01-10    1633.0      -12.0
                           N3ECAA   2017-01-15     753.0       -7.0
                           N3EDAA   2017-01-09    1450.0      -10.0
                                    2017-01-10     753.0       -7.0
                           N3ENAA   2017-01-24     756.0       -4.0
                                    2017-01-26    1533.0       33.0
...                                                  ...        ...
DL             XNA    ATL  N921AT   2017-01-20    1156.0       -3.0
                           N924DL   2017-01-30     555.0       -5.0
                           N925DL   2017-01-12     551.0       -9.0
                           N929AT   2017-01-08    1155.0       -4.0
                                    2017-01-31    1139.0      -20.0
                           N932AT   2017-01-12    1158.0       -1.0
                           N938AT   2017-01-26    1204.0        5.0
                           N940AT   2017-01-18    1157.0       -2.0
                                    2017-01-19    1200.0        1.0
                           N943DL   2017-01-22     555.0       -5.0
                           N950DL   2017-01-19     558.0       -2.0
                           N952DL   2017-01-18     556.0       -4.0
                           N953DL   2017-01-31     558.0       -2.0
                           N956DL   2017-01-17     554.0       -6.0
                           N961AT   2017-01-14    1233.0       -6.0
                           N964AT   2017-01-27    1155.0       -4.0
                           N966DL   2017-01-23     559.0       -1.0
                           N968DL   2017-01-29     555.0       -5.0
                           N969DL   2017-01-11     556.0       -4.0
                           N976DL   2017-01-09     622.0       22.0
                           N977AT   2017-01-24    1202.0        3.0
                                    2017-01-25    1149.0      -10.0
                           N977DL   2017-01-21     603.0       -2.0
                           N979AT   2017-01-15    1238.0       -1.0
                                    2017-01-22    1155.0       -4.0
                           N983AT   2017-01-11    1148.0      -11.0
                           N988DL   

In [97]:
hdf.loc[(['AA', 'DL', 'US'], ['ORD', 'DSM']), ['dep_time', 'dep_delay']]

dep_time  dep_delay
unique_carrier origin dest tail_num fl_date                        
AA             DSM    DFW  N424AA   2017-01-23    1324.0       -3.0
                           N426AA   2017-01-25     541.0       -9.0
                           N437AA   2017-01-13     542.0       -8.0
                                    2017-01-23     544.0       -6.0
                           N438AA   2017-01-11     542.0       -8.0
                           N439AA   2017-01-24     544.0       -6.0
                                    2017-01-31     544.0       -6.0
                           N4UBAA   2017-01-18    1323.0       -4.0
                           N4WNAA   2017-01-27    1322.0       -5.0
                           N4XBAA   2017-01-09     536.0      -14.0
                           N4XEAA   2017-01-21     544.0       -6.0
                           N4XFAA   2017-01-31    1320.0       -7.0
                           N4XGAA   2017-01-28    1337.0       10.0
                                    2017-01-30     542.0       -8.0
                           N4XJAA   2017-01-20     552.0        2.0
                                    2017-01-21    1320.0       -7.0
                           N4XKAA   2017-01-26    1323.0       -4.0
                           N4XMAA   2017-01-16    1423.0       56.0
                                    2017-01-19    1321.0       -6.0
                           N4XPAA   2017-01-09    1322.0       -5.0
                                    2017-01-14     545.0       -5.0
                           N4XTAA   2017-01-10    1355.0       28.0
                           N4XUAA   2017-01-13    1330.0        3.0
                                    2017-01-14    1319.0       -8.0
                           N4XVAA   2017-01-28       NaN        NaN
                           N4XXAA   2017-01-15    1322.0       -5.0
                                    2017-01-16     545.0       -5.0
                           N4XYAA   2017-01-18     559.0        9.0
                           N4YCAA   2017-01-26     545.0       -5.0
                                    2017-01-27     544.0       -6.0
...                                                  ...        ...
DL             ORD    SLC  N316NB   2017-01-23    1332.0       -6.0
                           N317NB   2017-01-09    1330.0       -8.0
                                    2017-01-11    1345.0        7.0
                           N319NB   2017-01-17    1353.0       15.0
                                    2017-01-22    1331.0       -7.0
                           N320NB   2017-01-13    1332.0       -6.0
                           N321NB   2017-01-19    1419.0       41.0
                           N323NB   2017-01-01    1732.0       57.0
                                    2017-01-02    1351.0       11.0
                           N324NB   2017-01-16    1337.0       -1.0
                           N326NB   2017-01-24    1332.0       -6.0
                                    2017-01-26    1349.0       11.0
                           N329NB   2017-01-06    1422.0       32.0
                           N330NB   2017-01-04    1344.0       -6.0
                                    2017-01-12    1343.0        5.0
                           N335NB   2017-01-31    1336.0       -2.0
                           N338NB   2017-01-29    1355.0       17.0
                           N347NB   2017-01-08    1338.0        0.0
                           N348NB   2017-01-10    1355.0       17.0
                           N349NB   2017-01-30    1333.0       -5.0
                           N352NW   2017-01-06    1857.0       10.0
                           N354NW   2017-01-04    1844.0       -3.0
                           N356NW   2017-01-02    1640.0       20.0
                           N358NW   2017-01-05    1856.0        9.0
                           N360NB   2017-01-25    1354.0       16.0
                           N365NB   2017-01-18    1350.0       12.0
                           N368NB   

In [102]:
hdf.loc[pd.IndexSlice[:, ['ORD', 'DSM']], ['dep_time', 'dep_delay']]

dep_time  dep_delay
unique_carrier origin dest tail_num fl_date                        
AA             DSM    DFW  N424AA   2017-01-23    1324.0       -3.0
                           N426AA   2017-01-25     541.0       -9.0
                           N437AA   2017-01-13     542.0       -8.0
                                    2017-01-23     544.0       -6.0
                           N438AA   2017-01-11     542.0       -8.0
                           N439AA   2017-01-24     544.0       -6.0
                                    2017-01-31     544.0       -6.0
                           N4UBAA   2017-01-18    1323.0       -4.0
                           N4WNAA   2017-01-27    1322.0       -5.0
                           N4XBAA   2017-01-09     536.0      -14.0
                           N4XEAA   2017-01-21     544.0       -6.0
                           N4XFAA   2017-01-31    1320.0       -7.0
                           N4XGAA   2017-01-28    1337.0       10.0
                                    2017-01-30     542.0       -8.0
                           N4XJAA   2017-01-20     552.0        2.0
                                    2017-01-21    1320.0       -7.0
                           N4XKAA   2017-01-26    1323.0       -4.0
                           N4XMAA   2017-01-16    1423.0       56.0
                                    2017-01-19    1321.0       -6.0
                           N4XPAA   2017-01-09    1322.0       -5.0
                                    2017-01-14     545.0       -5.0
                           N4XTAA   2017-01-10    1355.0       28.0
                           N4XUAA   2017-01-13    1330.0        3.0
                                    2017-01-14    1319.0       -8.0
                           N4XVAA   2017-01-28       NaN        NaN
                           N4XXAA   2017-01-15    1322.0       -5.0
                                    2017-01-16     545.0       -5.0
                           N4XYAA   2017-01-18     559.0        9.0
                           N4YCAA   2017-01-26     545.0       -5.0
                                    2017-01-27     544.0       -6.0
...                                                  ...        ...
WN             DSM    STL  N635SW   2017-01-15    1806.0        6.0
                           N645SW   2017-01-22    1800.0        0.0
                           N651SW   2017-01-01    1856.0       61.0
                           N654SW   2017-01-21    1156.0      126.0
                           N720WN   2017-01-23     605.0       -5.0
                                    2017-01-31     603.0       -7.0
                           N724SW   2017-01-30    1738.0       -7.0
                           N734SA   2017-01-20    1839.0       54.0
                           N737JW   2017-01-09     605.0       -5.0
                           N747SA   2017-01-27     610.0        0.0
                           N7718B   2017-01-18    1736.0       -9.0
                           N772SW   2017-01-31    1738.0       -7.0
                           N7735A   2017-01-11     603.0       -7.0
                           N773SA   2017-01-17    1743.0       -2.0
                           N7749B   2017-01-10    1746.0        1.0
                           N781WN   2017-01-02    1909.0       59.0
                                    2017-01-30     605.0       -5.0
                           N7827A   2017-01-14    1644.0      414.0
                           N7833A   2017-01-06     659.0       49.0
                           N7882B   2017-01-15     901.0        1.0
                           N791SW   2017-01-26    1744.0       -1.0
                           N903WN   2017-01-13    1908.0       83.0
                           N905WN   2017-01-05     605.0       -5.0
                           N944WN   2017-01-02     630.0        5.0
                           N949WN   2017-01-01     624.0        4.0
                           N952WN   2017-01-29     854.0       -6.0
                           N954WN   

In [103]:
pd.IndexSlice[:, ['ORD', 'DSM']]

(slice(None, None, None), ['ORD', 'DSM'])